In [1]:
import math
import numpy as np
from typing import List, Tuple
from enum import Enum
import xlrd
from decimal import Decimal, ROUND_DOWN, ROUND_UP

import direction_coefficient as dc
import f_value

# 変更履歴

- 2019.10.15: 浴室の床が断熱、浴室の床が外皮に接しない場合の計算を追加  
それにより、外皮の部位の面積の合計が異なることが判明した。（2019.10時点の評価方法においては、床断熱住戸については同じ面積を使用している。）

# 四捨五入

建研HPで公表する値としては、さらに小数点2桁までにまとめた値とする。  
丸める場合に、round関数は、JIS Z 8401 付属書A の方法（0.5の場合は偶数側に丸める）が適用されるため、自前に作成したround2関数で丸めることとする。この場合は、端数が「.5」の場合は、必ず大きい側（切り上げ側）に丸められる。

In [2]:
def round2(x,d=2):
    p=10**d
    return (x*p*2+1)//2/p
#    return x.quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)

# 住宅の種類

床・基礎部分の断熱方法

In [3]:
class FloorIns(Enum):

    # 床断熱
    FLOOR = 1
    
    # 基礎断熱
    BASE = 2

浴室における床・基礎部分の断熱方法

In [4]:
class BathIns(Enum):

    # （浴室が2階にある等）浴室床が外皮に面さない
    NOT_EXIST = 1
    
    # 床断熱
    FLOOR = 2
    
    # 基礎断熱
    BASE = 3

# 保持するパラメータ

In [5]:
class House():
    
    def __init__(self, name, ver, floor_ins, bath_ins):
        
        # 名前
        self.name = name
        
        # バージョン
        self.ver = ver

        # 床の断熱種類
        self.floor_ins = floor_ins
        
        # 浴室の断熱種類
        self.bath_ins = bath_ins
        
        # 床面積の合計, m2
        self.a_f_total = None
        
        # 形状係数
        self.sf = None
        
        # 1階の床面積に対する2階の床面積の比
        self.r_fa = None
        
        # 基礎の高さ（一般部分）, m
        self.h_base = None
        
        # 基礎の高さ（玄関）, m
        self.h_base_entrance = None
        
        # 基礎の高さ（浴室）, m
        self.h_base_bath = None
        
        # 高さ, m
        self.h = None
        
        # ドアの面積（玄関）, m2
        self.a_door_entrance = None
        
        # ドアの面積（勝手口）, m2
        self.a_door_backentrance = None
        
        # 玄関の土間床等の周辺部の基礎長さ（主開口方位側）, m
        self.l_base_entrance_ms = None
        
        # 玄関の土間床等の周辺部の基礎長さ（その他方位側）, m
        self.l_base_entrance_os = None
        
        # 浴室の土間床等の周辺部の基礎長さ（主開口方位側）, m
        self.l_base_bath_ms = None
        
        # 浴室の土間床等の周辺部の基礎長さ（その他方位側）, m
        self.l_base_bath_os = None
        
        # 開口部の面積比率
        self.r_opn = None
        
        # 方位別窓面積比率
        self.r_wnd = None
        
        # 床面積（土間床含む）, m2        
        self.a_f = None
        
        # 周長, m
        self.l_prm = None

        # 長辺の長さ, m
        self.l_ls = None
        
        # 短辺の長さ, m
        self.l_ss = None
        
        # 玄関の土間床等の面積, m2
        self.a_ef_entrance = None
        
        # 浴室の土間床等の面積, m2
        self.a_ef_bath = None
        
        # その他の土間床等の面積, m2
        self.a_ef_other = None
        
        # 浴室の床断熱した床の面積, m2
        self.a_if_bath = None
        
        # その他の床の面積, m2
        self.a_if_other = None
        
        # 屋根又は天井の面積, m2
        self.a_roof = None
        
        # 玄関等の土間床等の外周部の長さ（外気側）, m
        self.l_base_entrance_outside = None
        
        # 玄関等の土間床等の外周部の長さ（床下側）, m
        self.l_base_entrance_inside = None
        
        # 浴室の土間床等の外周部の長さ（外気側）, m
        self.l_base_bath_outside = None
        
        # 浴室の土間床等の外周部の長さ（床下側）, m
        self.l_base_bath_outside = None
        
        # その他の土間床等の外周部の長さ（外気側）, m
        self.l_base_other_outside = None
        
        # その他の土間床等の外周部の長さ（床下側）, m
        self.l_base_other_inside = None
        
        # 土間床周長（外気側）, m
        self.l_prm_outside = None
        
        # 土間床周長（床下側）, m
        self.l_prm_inside = None
        
        # 玄関等の基礎の面積（外気側）, m2
        self.a_base_entrance_outside = None
        
        # 玄関等の基礎の面積（床下側）, m2
        self.a_base_entrance_inside = None
        
        # 浴室の基礎の面積（外気側）, m2
        self.a_base_bath_outside = None
        
        # 浴室の基礎の面積（床下側）, m2
        self.a_base_bath_inside = None
        
        # その他の基礎の面積（外気側）, m2
        self.a_base_other_outside = None
        
        # その他の基礎の面積（床下側）, m2
        self.a_base_other_inside = None
        
        # 基礎の面積の合計
        self.a_base_total = None
        
        # 垂直外皮面積, m2
        self.a_srf = None
        
        # 基礎を除く外皮の部位の面積の合計, m2
        self.a_dash_env = None
        
        # 外皮の部位の面積の合計, m2
        self.a_env = None
        
        # 開口部の面積の合計, m2
        self.a_opn = None
        
        # 窓の面積の合計, m2
        self.a_wnd_total = None

        # 窓の面積, m2
        self.a_wnd = None
        
        # ドアの面積, m2
        self.a_door = None

        # 壁の面積, m2
        self.a_wall = None

# 計算ケース

以下のケースについて検討する。
- 床断熱住宅（外皮に接する浴室床なし）
- 床断熱住宅（浴室床断熱）
- 床断熱住宅（浴室基礎断熱）
- 基礎断熱住宅

In [6]:
hs = [
    House(name='床断熱住宅（外皮に接する浴室床なし）', ver=1, floor_ins=FloorIns.FLOOR, bath_ins=BathIns.NOT_EXIST),
    House(name='床断熱住宅（浴室床断熱）', ver=1, floor_ins=FloorIns.FLOOR, bath_ins=BathIns.FLOOR),
    House(name='床断熱住宅（浴室基礎断熱）', ver=1, floor_ins=FloorIns.FLOOR, bath_ins=BathIns.BASE),
    House(name='基礎断熱住宅', ver=1, floor_ins=FloorIns.BASE, bath_ins=None)
]

# 公表資料の値

In [7]:
docs = [
    {},
    {},
    {},
    {}
]

# 簡易評価式の算出の前提

部位の面積及び長さ等を計算するにあたっての前提条件を以下に示す。

| 意味 | 記号| 値 | 単位 | 備考 |
| --- | :---: | :---: | :---: | --- |
| 階数 | - | 2 | - | - |
| 床面積の合計 | $A_{F,total}$ | 90.0 | m<sup>2</sup> | - |
| 形状係数(1階) | $SF_1$ | 1.08 | - | - |
| 形状係数(2階) | $SF_2$ | 1.04 | - | - |
| 主開口方位 | - | 南西 | - | - |
| 1階の床面積に対する2階の床面積の比 | $r_{FA}$ | 0.77 | - | 自立循環型住宅プランの比を採用した。 |
| 基礎の高さ(基礎断熱住宅) | $h_{base}$ | 0.5 | m | 基礎の高さとは土間基礎上端から1階床レベルまでの間を言う。<br>(GLからではない。) |
| 基礎の高さ(玄関) | $h_{base,d}$ | 0.18 | m | - |
| 基礎の高さ(浴室) | $h_{base,b}$ | 0.5 | m | - |
| 高さ(1階) | $h_1$ | 2.9 | m | - |
| 高さ(2階) | $h_2$ | 2.7 | m | - |
| ドア(玄関) | $A_{door,1}$ | 1.89 | m<sup>2</sup> | 方位：北西 自立循環型住宅プランの面積を採用 |
| ドア(勝手口) | $A_{door,2}$ | 1.62 | m<sup>2</sup> | 方位：北東 自立循環型住宅プランの面積を採用 |
| 玄関の土間床等の周辺部の基礎長さ(主開口方位側) | $L_{base,d,MS}$ | 1.365 | m | - |
| 玄関の土間床等の周辺部の基礎長さ(その他の方位側) | $L_{base,d,OS}$ | 1.82 | m | - |
| 浴室の土間床等の周辺部の基礎長さ(主開口方位側) | $L_{base,b,MS}$ | 1.82 | m | - |
| 浴室の土間床等の周辺部の基礎長さ(その他の方位側) | $L_{base,b,OS}$ | 1.82 | m | - |
| 開口部の面積比率 | $r_{opn}$ | 0.14 | - | 床断熱住宅と基礎断熱住宅とを同じ開口部面積にするために、<br>基礎面積を含めない外皮表面積に対する値とする。 |
| 方位別窓面積比率(南西) | $r_{wnd,SW}$ | 0.6862 | - | - |
| 方位別窓面積比率(北西) | $r_{wnd,NW}$ | 0.0721 | - | - |
| 方位別窓面積比率(北東) | $r_{wnd,NE}$ | 0.1097 | - | - |
| 方位別窓面積比率(南東) | $r_{wnd,SE}$ | 0.1320 | - | - |


※基礎の高さについて  
下記を想定すると、根太がけの場合は524～539、根太レスの場合は479～494  
・基礎の高さは400mm（参考：長期優良住宅）  
・土台は105mm又は10mm  
・根太は45mm（根太がけの場合のみ）  
・床荒板およびフローリングは24mm（12mm＋12mm）  
・GLから土間基礎上端までは50mm  
あるいは下記を想定すると、根太がけの場合は489～504、根太レスの場合は444～459  
・床下空間高さ（土間基礎上端から大引き下端まで）は330mmを確保（長期優良）  
・大引は90又は105mm  
・根太は45mm（根太がけの場合のみ）  
・床荒板およびフローリングは24mm（12mm＋12mm）  
以上を勘案して、500mmと決めた。

## 階数

階数は2階とする。

##  床面積の合計

$ A_{F,total} = 90.0 $ (m<sup>2</sup>)

In [8]:
for h in hs:
    h.a_f_total = 90.0

## 形状係数

1階: $ SF_1 = 1.08 $  
2階: $ SF_2 = 1.04 $

In [9]:
for h in hs:
    h.sf = [1.08, 1.04]

## 主開口方位

南西とする。

## 1階の床面積に対する2階の床面積の比

自立循環型住宅プランの比を採用する。

$ r_{FA} = 0.77 $

In [10]:
for h in hs:
    h.r_fa = 0.77

## 基礎の高さ

$ h_{base} = 0.5 $ (m) (一般部分)  

$ h_{base,entrance} = 0.18 $ (m) (玄関)

$ h_{base,bath} = 0.5 $ (m) (浴室)

In [11]:
for h in hs:
    h.h_base = 0.5
    h.h_base_entrance = 0.18
    h.h_base_bath = 0.5

## 高さ

$ h_1 = 2.9 $ (1階)  
$ h_2 = 2.7 $ (2階)

In [12]:
for h in hs:
    h.h = [2.9, 2.7]

## ドアの面積

玄関の方位は北西、勝手口の方位は北東とする。自立循環型住宅プランの面積を採用した。

$ A_{door,entrance} = 1.89 $ (m<sup>2</sup>) (玄関)  

$ A_{door,backentrance} = 1.62 $ (m<sup>2</sup>) (勝手口)

In [13]:
for h in hs:
    h.a_door_entrance = 1.89
    h.a_door_backentrance = 1.62

## 玄関の土間床等の周辺部の基礎長さ

$L_{base,entrance,MS}=1.365$(m)(主開口方位側)

$L_{base,entrance,OS}=1.82$(m)(その他方位側)

In [14]:
for h in hs:
    h.l_base_entrance_ms = 1.365
    h.l_base_entrance_os = 1.82

## 浴室の土間床等の周辺部の基礎長さ

$L_{base,bath,MS}=1.82$(m)(主開口方位側)

$L_{base,bath,OS}=1.82$(m)(その他方位側)

In [15]:
for h in hs:
    h.l_base_bath_ms = 1.82
    h.l_base_bath_os = 1.82

## 開口部の面積比率

$r_{opn} = 0.12$

床断熱住宅と基礎断熱住宅とを同じ開口部面積にするために、基礎面積を含めない外皮表面積に対する値とする。

In [16]:
for h in hs:
    h.r_opn = 0.12

## 方位別窓面積比率

南西  
$r_{wnd,SW}=0.6862$  
北西  
$r_{wnd,NW}=0.0721$  
北東  
$r_{wnd,NE}=0.1097$  
南東  
$r_{wnd,SE}=0.1320$

In [17]:
for h in hs:
    h.r_wnd = [0.6862, 0.0721, 0.1097, 0.1320]

# 算出過程

In [18]:
def show_result(prm):
    for h in hs:
        print(str(h.name) + ' : ' + str(prm(h)))

## 床面積(土間床含む)

$ A_{F,1} = \frac{A_{F,total}}{ ( 1 + r_{FA} )} $  

$ A_{F,2} = A_{F,total} - A_{F,1} $

$A_{F,total}$: 床面積の合計(m<sup>2</sup>)  
$r_{FA}$: 1階の床面積に対する2階の床面積の比  
$A_{F,1}$: 1階の床面積(土間床含む)(m<sup>2</sup>)  
$A_{F,2}$: 2階の床面積(m<sup>2</sup>)

In [19]:
for h in hs:
    a_f_1 = h.a_f_total / (1.0 + h.r_fa)
    a_f_2 = h.a_f_total - a_f_1
    h.a_f = [a_f_1, a_f_2]

In [20]:
show_result(lambda h: h.a_f)

床断熱住宅（外皮に接する浴室床なし） : [50.847457627118644, 39.152542372881356]
床断熱住宅（浴室床断熱） : [50.847457627118644, 39.152542372881356]
床断熱住宅（浴室基礎断熱） : [50.847457627118644, 39.152542372881356]
基礎断熱住宅 : [50.847457627118644, 39.152542372881356]


## 周長

$ \begin{align} L_{prm,i} = \sqrt{A_{F,i}} \times 4 \times SF_i \end{align} $

$L_{prm,i}$: $i$階の周長(m)($i=1,2$)  
$SF_i$: $i$階の形状係数($i=1,2$)  

In [21]:
for h in hs:
    h.l_prm = [4 * sf_i * np.sqrt(a_f_i) for a_f_i, sf_i in zip(h.a_f, h.sf)]

In [22]:
show_result(lambda h: h.l_prm)

床断熱住宅（外皮に接する浴室床なし） : [30.80479821749104, 26.029948852968104]
床断熱住宅（浴室床断熱） : [30.80479821749104, 26.029948852968104]
床断熱住宅（浴室基礎断熱） : [30.80479821749104, 26.029948852968104]
基礎断熱住宅 : [30.80479821749104, 26.029948852968104]


## 長辺の長さ

$ \displaystyle L_{LS,i} = \frac{ \frac{L_{prm,i}}{2} + \sqrt{ \left( \frac{L_{prm,i}}{2} \right)^2 - 4 \times A_{F,i} } }{2} $

$L_{LS,i}$: $i$階の長辺の長さ($i=1,2$)

In [23]:
for h in hs:
    h.l_ls = [(l_prm_i / 2 + np.sqrt((l_prm_i / 2)**2 - 4 * a_f_i)) / 2
              for l_prm_i, a_f_i in zip(h.l_prm, h.a_f)]

In [24]:
show_result(lambda h: h.l_ls)

床断熱住宅（外皮に接する浴室床なし） : [10.609982280729707, 8.29490083566609]
床断熱住宅（浴室床断熱） : [10.609982280729707, 8.29490083566609]
床断熱住宅（浴室基礎断熱） : [10.609982280729707, 8.29490083566609]
基礎断熱住宅 : [10.609982280729707, 8.29490083566609]


## 短辺の長さ

$ \displaystyle L_{SS,i} = \frac{L_{prm,i}}{2} - L_{LS,i} $

$L_{ss,i}$: $i$階の短辺の長さ(m)

In [25]:
for h in hs:
    h.l_ss = [l_prm_i / 2 - l_ls_i for l_prm_i, l_ls_i in zip(h.l_prm, h.l_ls)]

In [26]:
show_result(lambda h: h.l_ss)

床断熱住宅（外皮に接する浴室床なし） : [4.7924168280158135, 4.720073590817963]
床断熱住宅（浴室床断熱） : [4.7924168280158135, 4.720073590817963]
床断熱住宅（浴室基礎断熱） : [4.7924168280158135, 4.720073590817963]
基礎断熱住宅 : [4.7924168280158135, 4.720073590817963]


## 玄関の土間床等の面積

$ A_{EF,entrance} = L_{base,entrance,MS} \times L_{base,entrance,OS} $

$A_{EF,entrance}$: 玄関の土間床等の面積(m<sup>2</sup>)  
$L_{base,entrance,MS}$: 玄関の土間床等の周辺部の基礎長さ(主開口方位側)(m)  
$L_{base,entrance,OS}$: 玄関の土間床等の周辺部の基礎長さ(その他の方位側)(m)  

In [27]:
for h in hs:
    h.a_ef_entrance = h.l_base_entrance_ms * h.l_base_entrance_os

In [28]:
show_result(lambda h: h.a_ef_entrance)

床断熱住宅（外皮に接する浴室床なし） : 2.4843
床断熱住宅（浴室床断熱） : 2.4843
床断熱住宅（浴室基礎断熱） : 2.4843
基礎断熱住宅 : 2.4843


## 浴室の土間床等の面積

$ A_{EF,bath} = L_{base,bath,MS} \times L_{base,bath,OS} $

$A_{EF,bath}$: 浴室の土間床等の面積(m<sup>2</sup>)  
$L_{base,bath,MS}$: 浴室の土間床等の周辺部の基礎長さ(主開口方位側)(m)  
$L_{base,bath,OS}$: 浴室の土間床等の周辺部の基礎長さ(その他の方位側)(m)  

ただし、「床断熱住宅（外皮に接する浴室床なし）」及び「床断熱住宅（浴室床断熱）」の場合は、0.0(m<sup>2</sup>)とする。

In [29]:
for h in hs:
    if h.floor_ins == FloorIns.FLOOR:
        if h.bath_ins == BathIns.NOT_EXIST:
            h.a_ef_bath = 0.0
        elif h.bath_ins == BathIns.FLOOR:
            h.a_ef_bath = 0.0
        elif h.bath_ins == BathIns.BASE:
            h.a_ef_bath = h.l_base_bath_ms * h.l_base_bath_os
        else:
            raise ValueError()
    elif h.floor_ins == FloorIns.BASE:
        h.a_ef_bath = h.l_base_bath_ms * h.l_base_bath_os
    else:
        raise ValueError()

In [30]:
show_result(lambda h: h.a_ef_bath)

床断熱住宅（外皮に接する浴室床なし） : 0.0
床断熱住宅（浴室床断熱） : 0.0
床断熱住宅（浴室基礎断熱） : 3.3124000000000002
基礎断熱住宅 : 3.3124000000000002


## その他の土間床等の面積

$ A_{EF,other} = A_{F,1} - A_{EF,entrance} - A_{EF,bath} $

$ A_{EF,other} $: 土間床等の面積(玄関・浴室除く)(m<sup>2</sup>)  

ただし、「床断熱住宅」の場合は、0.0(m<sup>2</sup>)とする。

In [31]:
for h in hs:
    if h.floor_ins == FloorIns.FLOOR:
        h.a_ef_other = 0.0
    elif h.floor_ins == FloorIns.BASE:
        h.a_ef_other = h.a_f[0] - h.a_ef_entrance - h.a_ef_bath
    else:
        raise ValueError()

In [32]:
show_result(lambda h: h.a_ef_other)

床断熱住宅（外皮に接する浴室床なし） : 0.0
床断熱住宅（浴室床断熱） : 0.0
床断熱住宅（浴室基礎断熱） : 0.0
基礎断熱住宅 : 45.05075762711864


## 浴室の床の面積（2018.4～）

2018年4月まで、床断熱住戸において浴室は必ず基礎断熱と想定していたため、2018年4月まではこの項目はなかった。

$ A_{IF,bath} = L_{base,bath,MS} \times L_{base,bath,OS} $

$A_{IF,bath}$: 浴室の床断熱した床の面積(m<sup>2</sup>)  
$L_{base,bath,MS}$: 浴室の土間床等の周辺部の基礎長さ(主開口方位側)(m)  
$L_{base,bath,OS}$: 浴室の土間床等の周辺部の基礎長さ(その他の方位側)(m)  

ただし、「床断熱住宅（外皮に接する浴室床なし）」及び「床断熱住宅（浴室床断熱）」の場合のみ。  
その他の場合は、0.0(m<sup>2</sup>)とする。

In [33]:
for h in hs:
    if h.floor_ins == FloorIns.FLOOR:
        if h.bath_ins == BathIns.NOT_EXIST:
            h.a_if_bath = h.l_base_bath_ms * h.l_base_bath_os
        elif h.bath_ins == BathIns.FLOOR:
            h.a_if_bath = h.l_base_bath_ms * h.l_base_bath_os
        elif h.bath_ins == BathIns.BASE:
            h.a_if_bath = 0.0
        else:
            raise ValueError()
    elif h.floor_ins == FloorIns.BASE:
        h.a_if_bath = 0.0
    else:
        raise ValueError()

In [34]:
show_result(lambda h: h.a_if_bath)

床断熱住宅（外皮に接する浴室床なし） : 3.3124000000000002
床断熱住宅（浴室床断熱） : 3.3124000000000002
床断熱住宅（浴室基礎断熱） : 0.0
基礎断熱住宅 : 0.0


## その他の床の面積

2018年4月までは、「床断熱した床の面積」と呼んでいた。

$ A_{IF} = A_{F,1} - A_{EF,entrane} - A_{EF,bath} - A_{EF,other} - A_{IF,bath} $ 

$A_{IF}$: 床断熱した床の面積(m<sup>2</sup>)

In [35]:
for h in hs:
    h.a_if_other = h.a_f[0] - h.a_ef_entrance - h.a_ef_bath - h.a_ef_other - h.a_if_bath

In [36]:
show_result(lambda h: h.a_if_other)

床断熱住宅（外皮に接する浴室床なし） : 45.05075762711864
床断熱住宅（浴室床断熱） : 45.05075762711864
床断熱住宅（浴室基礎断熱） : 45.05075762711864
基礎断熱住宅 : 0.0


## 屋根又は天井

天井・屋根(下屋を含む)の面積は、オーバーハング床を想定しない限り、1階の床面積(土間床含む)と等しい。

$ A_{roof} = A_{F,1}  $ 

$A_{roof}$: 天井・屋根(下屋を含む)の面積 (m<sup>2</sup>)

In [37]:
for h in hs:
    h.a_roof = h.a_f[0]

In [38]:
show_result(lambda h: h.a_roof)

床断熱住宅（外皮に接する浴室床なし） : 50.847457627118644
床断熱住宅（浴室床断熱） : 50.847457627118644
床断熱住宅（浴室基礎断熱） : 50.847457627118644
基礎断熱住宅 : 50.847457627118644


## 玄関等の土間床等の外周部の長さ（2017.7～）

2017年7月から。それまでは玄関とその他の基礎の区別がなかった。

外気側：  
南西: $L_{base,entrance,SW}=0.0$  
北西: $L_{base,entrance,NW}=L_{base,entrance,OS}$  
北東: $L_{base,entrance,NE}=L_{base,entrance,MS}$  
南東: $L_{base,entrance,SE}=0.0$  

床下側:  
$
L_{base,entrance,inside} = \left \{
\begin{array}{ll}
    L_{base,entrance,MS} + L_{base,entrance,OS} & \text{ ( floor insulted house ) } \\
    0.0 & \text{ ( insulated house ) }
\end{array}
\right.
$

In [39]:
for h in hs:
    h.l_base_entrance_outside = (0.0, h.l_base_entrance_os, h.l_base_entrance_ms, 0.0)
    h.l_base_entrance_inside = {
        FloorIns.FLOOR: h.l_base_entrance_ms + h.l_base_entrance_os,
        FloorIns.BASE: 0.0
    }[h.floor_ins]

In [40]:
show_result(lambda h: h.l_base_entrance_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 1.82, 1.365, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 1.82, 1.365, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 1.82, 1.365, 0.0)
基礎断熱住宅 : (0.0, 1.82, 1.365, 0.0)


In [41]:
show_result(lambda h: h.l_base_entrance_inside)

床断熱住宅（外皮に接する浴室床なし） : 3.185
床断熱住宅（浴室床断熱） : 3.185
床断熱住宅（浴室基礎断熱） : 3.185
基礎断熱住宅 : 0.0


## 浴室の土間床等の外周部の長さ（2018.4～）

外気側：  
南西:  $L_{base,bath,SW}=0.0$  
北西:  $L_{base,bath,NW}=L_{base,bath,OS}$  
北東:  $L_{base,bath,NE}=L_{base,bath,MS}$  
南東:  $L_{base,bath,SE}=0.0$

ただし、「床断熱住宅（外皮に接する浴室床なし）」、「床断熱住宅（浴室床断熱）」及び「基礎断熱住宅」の場合はいずれの長さも0mとする。

室内側:  
$ L_{base,bath,inside} = L_{base,bath,MS} + L_{base,bath,OS} $

ただし、「床断熱住宅（外皮に接する浴室床なし）」、「床断熱住宅（浴室床断熱）」及び「基礎断熱住宅」の場合はいずれの長さも0mとする。

In [42]:
for h in hs:
    if h.floor_ins == FloorIns.FLOOR:
        if h.bath_ins == BathIns.NOT_EXIST or h.bath_ins == BathIns.FLOOR:
            h.l_base_bath_outside = 0.0, 0.0, 0.0, 0.0
            h.l_base_bath_inside = 0.0
        elif h.bath_ins == BathIns.BASE:
            h.l_base_bath_outside = 0.0, h.l_base_bath_os, h.l_base_bath_ms, 0.0
            h.l_base_bath_inside = h.l_base_bath_ms + h.l_base_bath_os
        else:
            raise ValueError
    elif h.floor_ins == FloorIns.BASE:
        h.l_base_bath_outside = 0.0, h.l_base_bath_os, h.l_base_bath_ms, 0.0
        h.l_base_bath_inside = 0.0
    else:
        raise ValueError

In [43]:
show_result(lambda h: h.l_base_bath_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 1.82, 1.82, 0.0)
基礎断熱住宅 : (0.0, 1.82, 1.82, 0.0)


In [44]:
show_result(lambda h: h.l_base_bath_inside)

床断熱住宅（外皮に接する浴室床なし） : 0.0
床断熱住宅（浴室床断熱） : 0.0
床断熱住宅（浴室基礎断熱） : 3.64
基礎断熱住宅 : 0.0


## その他の土間床等の外周部の長さ（2018.4～）

南西: $L_{base,SW} = \left \{
\begin{array}{ll}
    0.0 & \text{ ( floor insulated house ) } \\
    L_{LS,1} - L_{base,entrance,SW} - L_{base,bath,SW} & \text{ ( base insulated house ) }
\end{array}
\right.
$

北西: $L_{base,NW} = \left \{
\begin{array}{ll}
    0.0 & \text{ ( floor insulated house ) } \\
    L_{SS,1} - L_{base,entrance,NW} - L_{base,bath,NW} & \text{ ( base insulated house ) }
\end{array}
\right.
$

北東: $L_{base,NE} = \left \{
\begin{array}{ll}
    0.0 & \text{ ( floor insulated house ) } \\
    L_{LS,1} - L_{base,entrance,NE} - L_{base,bath,NE} & \text{ ( base insulated house ) }
\end{array}
\right.
$

南東: $L_{base,SE} = \left \{
\begin{array}{ll}
    0.0 & \text{ ( floor insulated house ) } \\
    L_{SS,1} - L_{base,entrance,SE} - L_{base,bath,SE} & \text{ ( base insulated house ) }
\end{array}
\right.
$

室内側:  
$ L_{base,inside} = 0.0 $

In [45]:
for h in hs:

    if h.floor_ins == FloorIns.FLOOR:
        h.l_base_other_outside = (0.0, 0.0, 0.0, 0.0)
    elif h.floor_ins == FloorIns.BASE:
        h.l_base_other_outside = (
            h.l_ls[0] - h.l_base_entrance_outside[0] - h.l_base_bath_outside[0],
            h.l_ss[0] - h.l_base_entrance_outside[1] - h.l_base_bath_outside[1],
            h.l_ls[0] - h.l_base_entrance_outside[2] - h.l_base_bath_outside[2],
            h.l_ss[0] - h.l_base_entrance_outside[3] - h.l_base_bath_outside[3]
        )
    else:
        raise ValueError()    
    h.l_base_other_inside = 0.0

In [46]:
show_result(lambda h: h.l_base_other_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 0.0, 0.0, 0.0)
基礎断熱住宅 : (10.609982280729707, 1.1524168280158131, 7.424982280729706, 4.7924168280158135)


In [47]:
show_result(lambda h: h.l_base_other_inside)

床断熱住宅（外皮に接する浴室床なし） : 0.0
床断熱住宅（浴室床断熱） : 0.0
床断熱住宅（浴室基礎断熱） : 0.0
基礎断熱住宅 : 0.0


## 玄関等の基礎の面積（2017.7～）

南西:  $ A_{base,entrance,SW} = L_{base,entrance,SW} \times h_{base,entrance} $  
北西:  $ A_{base,entrance,NW} = L_{base,entrance,NW} \times h_{base,entrance} $  
北東:  $ A_{base,entrance,NE} = L_{base,entrance,NE} \times h_{base,entrance} $  
南東:  $ L_{base,entrance,SE} = L_{base,entrance,SE} \times h_{base,entrance} $  

$A_{base,entrance,SW}$: 基礎(玄関)(南西)の面積(m<sup>2</sup>)  
$A_{base,entrance,NW}$: 基礎(玄関)(北西)の面積(m<sup>2</sup>)  
$A_{base,entrance,NE}$: 基礎(玄関)(北東)の面積(m<sup>2</sup>)  
$A_{base,entrance,SE}$: 基礎(玄関)(南東)の面積(m<sup>2</sup>)  

室内側:  $ L_{base,entrance,inside} = L_{base,entrance,inside} \times h_{base,entrance} $

$A_{base,entrance,inside}$: 基礎(玄関)(室内側)の面積(m<sup>2</sup>)  

In [48]:
for h in hs:
    h.a_base_entrance_outside = (
        h.l_base_entrance_outside[0] * h.h_base_entrance,
        h.l_base_entrance_outside[1] * h.h_base_entrance,
        h.l_base_entrance_outside[2] * h.h_base_entrance,
        h.l_base_entrance_outside[3] * h.h_base_entrance
    )    
    h.a_base_entrance_inside = h.l_base_entrance_inside * h.h_base_entrance

In [49]:
show_result(lambda h: h.a_base_entrance_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 0.3276, 0.2457, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 0.3276, 0.2457, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 0.3276, 0.2457, 0.0)
基礎断熱住宅 : (0.0, 0.3276, 0.2457, 0.0)


In [50]:
show_result(lambda h: h.a_base_entrance_inside)

床断熱住宅（外皮に接する浴室床なし） : 0.5733
床断熱住宅（浴室床断熱） : 0.5733
床断熱住宅（浴室基礎断熱） : 0.5733
基礎断熱住宅 : 0.0


## 浴室の基礎の面積（2018.4～）

南西:  $ A_{base,bath,SW} = L_{base,bath,SW} \times h_{base,bath} $  
北西:  $ A_{base,bath,NW} = L_{base,bath,NW} \times h_{base,bath} $  
北東:  $ A_{base,bath,NE} = L_{base,bath,NE} \times h_{base,bath} $  
南東:  $ L_{base,bath,SE} = L_{base,bath,SE} \times h_{base,bath} $  

$A_{base,bath,SW}$: 基礎(浴室)(南西)の面積(m<sup>2</sup>)  
$A_{base,bath,NW}$: 基礎(浴室)(北西)の面積(m<sup>2</sup>)  
$A_{base,bath,NE}$: 基礎(浴室)(北東)の面積(m<sup>2</sup>)  
$A_{base,bath,SE}$: 基礎(浴室)(南東)の面積(m<sup>2</sup>)  

室内側:  $ L_{base,bath,inside} = L_{base,bath,inside} \times h_{base,bath} $

$A_{base,bath,inside}$: 基礎(浴室)(室内側)の面積(m<sup>2</sup>)  

In [51]:
for h in hs:
    h.a_base_bath_outside = (
        h.l_base_bath_outside[0] * h.h_base_bath,
        h.l_base_bath_outside[1] * h.h_base_bath,
        h.l_base_bath_outside[2] * h.h_base_bath,
        h.l_base_bath_outside[3] * h.h_base_bath
    )
    h.a_base_bath_inside = h.l_base_bath_inside * h.h_base_bath

In [52]:
show_result(lambda h: h.a_base_bath_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 0.91, 0.91, 0.0)
基礎断熱住宅 : (0.0, 0.91, 0.91, 0.0)


In [53]:
show_result(lambda h: h.a_base_bath_inside)

床断熱住宅（外皮に接する浴室床なし） : 0.0
床断熱住宅（浴室床断熱） : 0.0
床断熱住宅（浴室基礎断熱） : 1.82
基礎断熱住宅 : 0.0


## その他の基礎の面積

ここで一旦その他の基礎の面積を求めるが、この値を公表値として使用するのではなく、この値を用いて基礎の面積の合計を求め、四捨五入した基礎の面積の合計から、四捨五入した玄関等の基礎の面積、四捨五入した浴室の基礎の面積を減じることで、その他の基礎の面積を求めている。（2020年に改善予定）

南西: $ A_{base,other,SW} = L_{base,SW} \times h_{base} $  
北西: $ A_{base,other,NW} = L_{base,NW} \times h_{base} $  
北東: $ A_{base,other,NE} = L_{base,NE} \times h_{base} $  
南東: $ A_{base,other,SE} = L_{base,SE} \times h_{base} $  

$A_{base,other,SW}$: 基礎断熱部分(玄関・浴室除く)(南西)の面積(m<sup>2</sup>)  
$A_{base,other,NW}$: 基礎断熱部分(玄関・浴室除く)(北西)の面積(m<sup>2</sup>)  
$A_{base,other,NE}$: 基礎断熱部分(玄関・浴室除く)(北東)の面積(m<sup>2</sup>)  
$A_{base,other,SE}$: 基礎断熱部分(玄関・浴室除く)(南東)の面積(m<sup>2</sup>)  

室内側:  $ A_{base,other,inside} = 0.0 $

$A_{base,other,inside}$: 基礎断熱部分(玄関・浴室除く)(室内側)の面積(m<sup>2</sup>)  

In [54]:
for h in hs:
    h.a_base_other_outside = (
        h.l_base_other_outside[0] * h.h_base,
        h.l_base_other_outside[1] * h.h_base,
        h.l_base_other_outside[2] * h.h_base,
        h.l_base_other_outside[3] * h.h_base
    )
    h.a_base_other_inside = 0.0

In [55]:
show_result(lambda h: h.a_base_other_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 0.0, 0.0, 0.0)
基礎断熱住宅 : (5.304991140364853, 0.5762084140079066, 3.712491140364853, 2.3962084140079067)


In [56]:
show_result(lambda h: h.a_base_other_inside)

床断熱住宅（外皮に接する浴室床なし） : 0.0
床断熱住宅（浴室床断熱） : 0.0
床断熱住宅（浴室基礎断熱） : 0.0
基礎断熱住宅 : 0.0


## 基礎断熱部分の面積の合計

$ A_{base,total} = A_{base,other,SW} + A_{base,other,NW} + A_{base,other,NE} + A_{base,other,SE} + A_{base,other,inside}\\
+ A_{base,entrance,SW} + A_{base,entrance,NW} + A_{base,entrance,NE} + A_{base,entrance,SE} + A_{base,entrance,inside} \\
+ A_{base,bath,SW} + A_{base,bath,NW} + A_{base,bath,NE} + A_{base,bath,SE} + A_{base,bath,inside} $

$A_{base,total}$: 基礎の面積の合計(m<sup>2</sup>)  

In [57]:
for h in hs:
    h.a_base_total = sum(h.a_base_other_outside) + h.a_base_other_inside \
        + sum(h.a_base_entrance_outside) + h.a_base_entrance_inside \
        + sum(h.a_base_bath_outside) + h.a_base_bath_inside

In [58]:
show_result(lambda h: h.a_base_total)

床断熱住宅（外皮に接する浴室床なし） : 1.1466
床断熱住宅（浴室床断熱） : 1.1466
床断熱住宅（浴室基礎断熱） : 4.7866
基礎断熱住宅 : 14.38319910874552


## 壁面積(開口部含む)

1階:  
南西: $ A_{srf,1,SW} = L_{LS,1} \times h_1 $  
北西: $ A_{srf,1,NW} = L_{SS,1} \times h_1 $  
北東: $ A_{srf,1,NE} = L_{LS,1} \times h_1 $  
北西: $ A_{srf,1,SE} = L_{SS,1} \times h_1 $

2階:  
南西: $ A_{srf,2,SW} = L_{LS,2} \times h_2 $  
北西: $ A_{srf,2,NW} = L_{SS,2} \times h_2 $  
北東: $ A_{srf,2,NE} = L_{LS,2} \times h_2 $  
北西: $ A_{srf,2,SE} = L_{SS,2} \times h_2 $

In [59]:
for h in hs:
    h.a_srf = [
        (h.l_ls[0] * h.h[0], h.l_ss[0] * h.h[0], h.l_ls[0] * h.h[0], h.l_ss[0] * h.h[0]),
        (h.l_ls[1] * h.h[1], h.l_ss[1] * h.h[1], h.l_ls[1] * h.h[1], h.l_ss[1] * h.h[1])
    ]

In [60]:
show_result(lambda h: h.a_srf)

床断熱住宅（外皮に接する浴室床なし） : [(30.768948614116148, 13.898008801245858, 30.768948614116148, 13.898008801245858), (22.396232256298443, 12.7441986952085, 22.396232256298443, 12.7441986952085)]
床断熱住宅（浴室床断熱） : [(30.768948614116148, 13.898008801245858, 30.768948614116148, 13.898008801245858), (22.396232256298443, 12.7441986952085, 22.396232256298443, 12.7441986952085)]
床断熱住宅（浴室基礎断熱） : [(30.768948614116148, 13.898008801245858, 30.768948614116148, 13.898008801245858), (22.396232256298443, 12.7441986952085, 22.396232256298443, 12.7441986952085)]
基礎断熱住宅 : [(30.768948614116148, 13.898008801245858, 30.768948614116148, 13.898008801245858), (22.396232256298443, 12.7441986952085, 22.396232256298443, 12.7441986952085)]


## 外皮面積の合計(基礎の面積を除く)

$ A'_{env} = A_{F,1} + A_{roof} + A_{srf,1,SW} + A_{srf,1,NW} + A_{srf,1,NE} + A_{srf,1,SE} + A_{srf,2,SW} + A_{srf,2,NW} + A_{srf,2,NE} + A_{srf,2,SE} $

$ A'_{env}$: 外皮面積の合計(基礎の面積を除く)(m<sup>2</sup>)

In [61]:
for h in hs:
    h.a_dash_env = h.a_f[0] + h.a_roof + sum(h.a_srf[0]) + sum(h.a_srf[1])

In [62]:
show_result(lambda h: h.a_dash_env)

床断熱住宅（外皮に接する浴室床なし） : 261.30969198797516
床断熱住宅（浴室床断熱） : 261.30969198797516
床断熱住宅（浴室基礎断熱） : 261.30969198797516
基礎断熱住宅 : 261.30969198797516


## 外皮面積の合計(m<sup>2</sup>)

$ A_{env} = A'_{env} + A_{base,total} $

$ A_{env} $: 外皮面積の合計(m<sup>2</sup>)

In [63]:
for h in hs:
    h.a_env = h.a_dash_env + h.a_base_total

In [64]:
show_result(lambda h: h.a_env)

床断熱住宅（外皮に接する浴室床なし） : 262.45629198797513
床断熱住宅（浴室床断熱） : 262.45629198797513
床断熱住宅（浴室基礎断熱） : 266.0962919879752
基礎断熱住宅 : 275.6928910967207


## 開口部の面積の合計

$ A_{opn} = A'_{env} \times r_{opn} $

$A_{opn}$: 開口部の面積の合計(m<sup>2</sup>)

In [65]:
for h in hs:
    h.a_opn = h.a_dash_env * h.r_opn

In [66]:
show_result(lambda h: h.a_opn)

床断熱住宅（外皮に接する浴室床なし） : 31.35716303855702
床断熱住宅（浴室床断熱） : 31.35716303855702
床断熱住宅（浴室基礎断熱） : 31.35716303855702
基礎断熱住宅 : 31.35716303855702


## 窓の面積の合計

$ A_{wnd,total} = A_{opn} - A_{door,entrance} - A_{door,backentrance} $

$A_{wnd,total}$: 窓の面積(m<sup>2</sup>)

In [67]:
for h in hs:
    h.a_wnd_total = h.a_opn - h.a_door_entrance - h.a_door_backentrance

In [68]:
show_result(lambda h: h.a_wnd_total)

床断熱住宅（外皮に接する浴室床なし） : 27.847163038557017
床断熱住宅（浴室床断熱） : 27.847163038557017
床断熱住宅（浴室基礎断熱） : 27.847163038557017
基礎断熱住宅 : 27.847163038557017


## 窓の面積

南西側: $ A_{wnd,SW} = A_{wnd} \times r_{wnd,SW} $  
北西側: $ A_{wnd,NW} = A_{wnd} \times r_{wnd,NW} $  
北東側: $ A_{wnd,NE} = A_{wnd} \times r_{wnd,NE} $  
南東側: $ A_{wnd,SE} = A_{wnd} \times r_{wnd,SE} $  

$ A_{wnd,SW}$: 窓の面積(南西)(m<sup>2</sup>)  
$ A_{wnd,NW}$: 窓の面積(北西)(m<sup>2</sup>)  
$ A_{wnd,NE}$: 窓の面積(北東)(m<sup>2</sup>)  
$ A_{wnd,SE}$: 窓の面積(南東)(m<sup>2</sup>)  

In [69]:
for h in hs:
    h.a_wnd = h.a_wnd_total * h.r_wnd[0], h.a_wnd_total * h.r_wnd[1], h.a_wnd_total * h.r_wnd[2], h.a_wnd_total * h.r_wnd[3]

In [70]:
show_result(lambda h: h.a_wnd)

床断熱住宅（外皮に接する浴室床なし） : (19.108723277057827, 2.0077804550799607, 3.054833785329705, 3.6758255210895263)
床断熱住宅（浴室床断熱） : (19.108723277057827, 2.0077804550799607, 3.054833785329705, 3.6758255210895263)
床断熱住宅（浴室基礎断熱） : (19.108723277057827, 2.0077804550799607, 3.054833785329705, 3.6758255210895263)
基礎断熱住宅 : (19.108723277057827, 2.0077804550799607, 3.054833785329705, 3.6758255210895263)


## ドアの面積

南西側: $ A_{door,SW} = 0.0 $  
北西側: $ A_{door,NW} = A_{door,entrance} $   
北東側: $ A_{door,NE} = A_{door,backentrance} $  
南東側: $ A_{door,SE} = 0.0 $  

$A_{door}$: ドアの面積(m<sup>2</sup>)

In [71]:
for h in hs:
    h.a_door = 0.0, h.a_door_entrance, h.a_door_backentrance, 0.0

In [72]:
show_result(lambda h: h.a_door_entrance)

床断熱住宅（外皮に接する浴室床なし） : 1.89
床断熱住宅（浴室床断熱） : 1.89
床断熱住宅（浴室基礎断熱） : 1.89
基礎断熱住宅 : 1.89


## 壁の面積

南西側: $ A_{wall,SW} = A_{srf,1,SW} + A_{srf,2,SW} - A_{wnd,SW} - A_{door,SW} $  
北西側: $ A_{wall,NW} = A_{srf,1,NW} + A_{srf,2,NW} - A_{wnd,NW} - A_{door,NW} $  
北東側: $ A_{wall,NE} = A_{srf,1,NE} + A_{srf,2,NE} - A_{wnd,NE} - A_{door,NE} $  
南東側: $ A_{wall,SE} = A_{srf,1,SE} + A_{srf,2,SE} - A_{wnd,SE} - A_{door,SE} $  

$ A_{wall}$: 壁の面積(m<sup>2</sup>)

In [73]:
for h in hs:
    h.a_wall = (
        h.a_srf[0][0] + h.a_srf[1][0] - h.a_wnd[0] - h.a_door[0],
        h.a_srf[0][1] + h.a_srf[1][1] - h.a_wnd[1] - h.a_door[1],
        h.a_srf[0][2] + h.a_srf[1][2] - h.a_wnd[2] - h.a_door[2],
        h.a_srf[0][3] + h.a_srf[1][3] - h.a_wnd[3] - h.a_door[3]
    )

In [74]:
show_result(lambda h: h.a_wall)

床断熱住宅（外皮に接する浴室床なし） : (34.05645759335676, 22.744427041374397, 48.490347085084885, 22.966381975364833)
床断熱住宅（浴室床断熱） : (34.05645759335676, 22.744427041374397, 48.490347085084885, 22.966381975364833)
床断熱住宅（浴室基礎断熱） : (34.05645759335676, 22.744427041374397, 48.490347085084885, 22.966381975364833)
基礎断熱住宅 : (34.05645759335676, 22.744427041374397, 48.490347085084885, 22.966381975364833)


## 熱橋（屋根と間仕切り等） T字（室2・外2）

長辺方向と短辺方向に十字に梁がはしっていると過程し、長辺方向の長さと短辺方向の長さを足した値とする。

$$ L_{HB,roof-iw,top} = L_{SS,1} + L_{LS,1} $$

In [75]:
for h in hs:
    h.l_hb_roof_iw_top = h.l_ss[0] + h.l_ls[0]

In [76]:
show_result(lambda h: h.l_hb_roof_iw_top)

床断熱住宅（外皮に接する浴室床なし） : 15.40239910874552
床断熱住宅（浴室床断熱） : 15.40239910874552
床断熱住宅（浴室基礎断熱） : 15.40239910874552
基礎断熱住宅 : 15.40239910874552


## 熱橋（屋根と外壁） L字（室1・外3）

1階は1階の周長とする。ただし、外壁と階床の熱橋長さを引くため、2階の周長の半分を減じる。外壁と階床の熱橋は南西と南東に面するものとする。  
2階は2階の周長とする。

南西と上に面した熱橋  
$$ L_{HB,roof-ew,top-SW} = \left( L_{LS,1} - L_{LS,2} \right) + L_{LS,2} $$  
北西と上に面した熱橋  
$$ L_{HB,roof-ew,top-NW} = L_{SS,1} + L_{SS,2} $$  
北東と上に面した熱橋  
$$ L_{HB,roof-ew,top-NE} = L_{LS,1} + L_{LS,2} $$  
南東と上に面した熱橋  
$$ L_{HB,roof-ew,top-SE} = \left( L_{SS,1} - L_{SS,2} \right) + L_{ss,2} $$  

In [77]:
for h in hs:
    h.l_hb_roof_ew_top_sw = h.l_ls[0] - h.l_ls[1] + h.l_ls[1]
    h.l_hb_roof_ew_top_nw = h.l_ss[0] + h.l_ss[1]
    h.l_hb_roof_ew_top_ne = h.l_ls[0] + h.l_ls[1]
    h.l_hb_roof_ew_top_se = h.l_ss[0] - h.l_ss[1] + h.l_ss[1]

In [78]:
show_result(lambda h: (h.l_hb_roof_ew_top_sw, h.l_hb_roof_ew_top_nw, h.l_hb_roof_ew_top_ne, h.l_hb_roof_ew_top_se))

床断熱住宅（外皮に接する浴室床なし） : (10.609982280729707, 9.512490418833776, 18.904883116395794, 4.7924168280158135)
床断熱住宅（浴室床断熱） : (10.609982280729707, 9.512490418833776, 18.904883116395794, 4.7924168280158135)
床断熱住宅（浴室基礎断熱） : (10.609982280729707, 9.512490418833776, 18.904883116395794, 4.7924168280158135)
基礎断熱住宅 : (10.609982280729707, 9.512490418833776, 18.904883116395794, 4.7924168280158135)


## 熱橋（屋根と外壁と階床） 十字（室3・外1）

2階の周長の半分とする。方位は北西側と北東側とする。

南西と上に面した熱橋  
$$ L_{HB,roof-ew-if,top-SW} = 0 $$  
北西と上に面した熱橋  
$$ L_{HB,roof-ew-if,top-NW} = L_{SS,2} $$  
北東と上に面した熱橋  
$$ L_{HB,roof-ew-if,top-NE} = L_{LS,2} $$  
南東と上に面した熱橋  
$$ L_{HB,roof-ew-if,top-SE} = 0 $$  

In [79]:
for h in hs:
    h.l_hb_roof_ew_if_top_sw = 0.0
    h.l_hb_roof_ew_if_top_nw = h.l_ss[1]
    h.l_hb_roof_ew_if_top_ne = h.l_ls[1]
    h.l_hb_roof_ew_if_top_se = 0.0

In [80]:
show_result(lambda h: (h.l_hb_roof_ew_if_top_sw, h.l_hb_roof_ew_if_top_nw, h.l_hb_roof_ew_if_top_ne, h.l_hb_roof_ew_if_top_se))

床断熱住宅（外皮に接する浴室床なし） : (0.0, 4.720073590817963, 8.29490083566609, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 4.720073590817963, 8.29490083566609, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 4.720073590817963, 8.29490083566609, 0.0)
基礎断熱住宅 : (0.0, 4.720073590817963, 8.29490083566609, 0.0)


## 熱橋（外壁と間仕切り等） T字（室2・外2）

外壁は間仕切りと4箇所で接続するため、建物高さの4倍とする。

南西に面した熱橋  
$$ L_{HB,ew-iw,SW} = h_1 + h_2 $$  
南西に面した熱橋  
$$ L_{HB,ew-iw,NW} = h_1 + h_2 $$  
南西に面した熱橋  
$$ L_{HB,ew-iw,NE} = h_1 + h_2 $$  
南西に面した熱橋  
$$ L_{HB,ew-iw,SE} = h_1 + h_2 $$  

In [81]:
for h in hs:
    h.l_hb_ew_iw_sw = h.h[0] + h.h[1]
    h.l_hb_ew_iw_nw = h.h[0] + h.h[1]
    h.l_hb_ew_iw_ne = h.h[0] + h.h[1]
    h.l_hb_ew_iw_se = h.h[0] + h.h[1]

In [82]:
show_result(lambda h: (h.l_hb_ew_iw_sw, h.l_hb_ew_iw_nw, h.l_hb_ew_iw_ne, h.l_hb_ew_iw_se))

床断熱住宅（外皮に接する浴室床なし） : (5.6, 5.6, 5.6, 5.6)
床断熱住宅（浴室床断熱） : (5.6, 5.6, 5.6, 5.6)
床断熱住宅（浴室基礎断熱） : (5.6, 5.6, 5.6, 5.6)
基礎断熱住宅 : (5.6, 5.6, 5.6, 5.6)


## 熱橋（外壁と階床） T字（室2・外2）

2階の周長の半分とし、南西と南東に発生するものとする。

南西に面した熱橋  
$$ L_{HB,ew-if,SW} = L_{LS,2} $$  
北西に面した熱橋  
$$ L_{HB,ew-if,NW} = 0 $$  
北東に面した熱橋  
$$ L_{HB,ew-if,NE} = 0 $$  
南東に面した熱橋  
$$ L_{HB,ew-if,SE} = L_{SS,2} $$

In [83]:
for h in hs:
    h.l_hb_ew_if_sw = h.l_ls[1]
    h.l_hb_ew_if_nw = 0.0
    h.l_hb_ew_if_ne = 0.0
    h.l_hb_ew_if_se = h.l_ss[1]

In [84]:
show_result(lambda h: (h.l_hb_ew_if_sw, h.l_hb_ew_if_nw, h.l_hb_ew_if_ne, h.l_hb_ew_if_se))

床断熱住宅（外皮に接する浴室床なし） : (8.29490083566609, 0.0, 0.0, 4.720073590817963)
床断熱住宅（浴室床断熱） : (8.29490083566609, 0.0, 0.0, 4.720073590817963)
床断熱住宅（浴室基礎断熱） : (8.29490083566609, 0.0, 0.0, 4.720073590817963)
基礎断熱住宅 : (8.29490083566609, 0.0, 0.0, 4.720073590817963)


## 熱橋（外壁と外壁 出隅） L字（室1・外3）

1階部分は4隅と玄関・浴室の2箇所の計6箇所とする。  
2階部分は4隅の4箇所とする。

南西と北西に面した熱橋  
$$ L_{HB,ew-ew,oc,SW-NW} = h_1 + h_2 $$  
北西と北東に面した熱橋  
$$ L_{HB,ew-ew,oc,NW-NE} = 3 h_1 + h_2 $$  
北東と南東に面した熱橋  
$$ L_{HB,ew-ew,oc,NE-SE} = h_1 + h_2 $$  
南東と南西に面した熱橋  
$$ L_{HB,ew-ew,oc,SE-SW} = h_1 + h_2 $$

In [85]:
for h in hs:
    h.l_hb_ew_ew_oc_sw_nw = h.h[0] + h.h[1]
    h.l_hb_ew_ew_oc_nw_ne = 3 * h.h[0] + h.h[1]
    h.l_hb_ew_ew_oc_ne_se = h.h[0] + h.h[1]
    h.l_hb_ew_ew_oc_se_sw = h.h[0] + h.h[1]

In [86]:
show_result(lambda h: (h.l_hb_ew_ew_oc_sw_nw, h.l_hb_ew_ew_oc_nw_ne, h.l_hb_ew_ew_oc_ne_se, h.l_hb_ew_ew_oc_se_sw))

床断熱住宅（外皮に接する浴室床なし） : (5.6, 11.399999999999999, 5.6, 5.6)
床断熱住宅（浴室床断熱） : (5.6, 11.399999999999999, 5.6, 5.6)
床断熱住宅（浴室基礎断熱） : (5.6, 11.399999999999999, 5.6, 5.6)
基礎断熱住宅 : (5.6, 11.399999999999999, 5.6, 5.6)


## 熱橋（外壁と外壁 入隅） L字（室3・外1）

1階において玄関と浴室の2箇所とする。北側の熱橋である。

南西と北西に面した熱橋  
$$ L_{HB,ew-ew,ic,SW-NW} = 0 $$  
北西と北東に面した熱橋  
$$ L_{HB,ew-ew,ic,NW-NE} = 2 h_1 $$  
北東と南東に面した熱橋  
$$ L_{HB,ew-ew,ic,NE-SE} = 0 $$  
南東と南西に面した熱橋  
$$ L_{HB,ew-ew,ic,SE-SW} = 0 $$  

In [87]:
for h in hs:
    h.l_hb_ew_ew_ic_sw_nw = 0.0
    h.l_hb_ew_ew_ic_nw_ne = 2 * h.h[0]
    h.l_hb_ew_ew_ic_ne_se = 0.0
    h.l_hb_ew_ew_ic_se_sw = 0.0

In [88]:
show_result(lambda h: (h.l_hb_ew_ew_ic_sw_nw, h.l_hb_ew_ew_ic_nw_ne, h.l_hb_ew_ew_ic_ne_se, h.l_hb_ew_ew_ic_se_sw))

床断熱住宅（外皮に接する浴室床なし） : (0.0, 5.8, 0.0, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 5.8, 0.0, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 5.8, 0.0, 0.0)
基礎断熱住宅 : (0.0, 5.8, 0.0, 0.0)


## 熱橋（床と間仕切り等） 十字（室2・外2）

長辺方向と短辺方向に十字に梁がはしっていると過程し、長辺方向の長さと短辺方向の長さを足した値とする。

下に面した熱橋  
$$ L_{HB,ef-iw,inside} = L_{SS,1} + L_{LS,1} $$

In [89]:
for h in hs:
    h.l_hb_ef_iw_inside = h.l_ss[0] + h.l_ls[0]

In [90]:
show_result(lambda h: h.l_hb_ef_iw_inside)

床断熱住宅（外皮に接する浴室床なし） : 15.40239910874552
床断熱住宅（浴室床断熱） : 15.40239910874552
床断熱住宅（浴室基礎断熱） : 15.40239910874552
基礎断熱住宅 : 15.40239910874552


## 熱橋（床と外壁と基礎） T字（床断熱時：室1・外3）（基礎断熱時：室2・外2）

1階の周長とする。ただし、玄関の外周部長さを減じる。  
浴室については、住宅の断熱境界にかかわらず何らかの熱橋が生じるものとし、減じない。

南西と床下に面した熱橋  
$$ L_{HB,ef-ew-base,SW-bottom} = L_{LS,1} - L_{base,entrance,SW} $$  
北西と床下に面した熱橋  
$$ L_{HB,ef-ew-base,NW-bottom} = L_{SS,1} - L_{base,entrance,NW} $$  
北東と床下に面した熱橋  
$$ L_{HB,ef-ew-base,NE-bottom} = L_{LS,1} - L_{base,entrance,NE} $$  
南東と床下に面した熱橋  
$$ L_{HB,ef-ew-base,SE-bottom} = L_{SS,1} - L_{base,entrance,SE} $$  

In [91]:
for h in hs:
    h.l_hb_ef_ew_base_sw_bottom = h.l_ls[0] - h.l_base_entrance_outside[0]
    h.l_hb_ef_ew_base_nw_bottom = h.l_ss[0] - h.l_base_entrance_outside[1]
    h.l_hb_ef_ew_base_ne_bottom = h.l_ls[0] - h.l_base_entrance_outside[2]
    h.l_hb_ef_ew_base_se_bottom = h.l_ss[0] - h.l_base_entrance_outside[3]

In [92]:
show_result(lambda h: (h.l_hb_ef_ew_base_sw_bottom, h.l_hb_ef_ew_base_nw_bottom, h.l_hb_ef_ew_base_ne_bottom, h.l_hb_ef_ew_base_se_bottom))

床断熱住宅（外皮に接する浴室床なし） : (10.609982280729707, 2.972416828015813, 9.244982280729706, 4.7924168280158135)
床断熱住宅（浴室床断熱） : (10.609982280729707, 2.972416828015813, 9.244982280729706, 4.7924168280158135)
床断熱住宅（浴室基礎断熱） : (10.609982280729707, 2.972416828015813, 9.244982280729706, 4.7924168280158135)
基礎断熱住宅 : (10.609982280729707, 2.972416828015813, 9.244982280729706, 4.7924168280158135)


## 熱橋（床と基礎） 玄関

玄関の基礎は無断熱を想定するため、玄関における熱橋（床と基礎）は存在しないとして扱う。

## 熱橋（床と基礎）浴室 L字又はT字（床断熱（浴室基礎断熱）時：室3・外1）（床断熱（浴室床断熱 or 浴室の床が外皮に接しない）時：室2・外2）

基礎断熱住宅は発生しないものとする。  

外気に面さない熱橋  
$$ L_{HB,ef-base,inside} = L_{base,bath,MS} + L_{base,bath,OS} $$  

In [93]:
for h in hs:
    if h.floor_ins == FloorIns.FLOOR:
        h.l_hb_ef_base_inside = h.l_base_bath_ms + h.l_base_bath_os
    elif h.floor_ins == FloorIns.BASE:
        h.l_hb_ef_base_inside = 0.0
    else:
        raise ValueError

In [94]:
show_result(lambda h: h.l_hb_ef_base_inside)

床断熱住宅（外皮に接する浴室床なし） : 3.64
床断熱住宅（浴室床断熱） : 3.64
床断熱住宅（浴室基礎断熱） : 3.64
基礎断熱住宅 : 0.0


## 熱橋（床と基礎）その他

間仕切り以外でも、床と基礎のT字部分で床の下側で断熱している場合は構造熱橋が発生するが、簡易法ではその部分の熱橋は扱わないことにする。

## グルーピングされた熱橋（屋根又は天井）

$$ L_{groupedHB,roof,top} = L_{HB,roof-iw,top} $$

In [95]:
for h in hs:
    h.l_groupedhb_roof_top = h.l_hb_roof_iw_top

In [96]:
show_result(lambda h: h.l_groupedhb_roof_top)

床断熱住宅（外皮に接する浴室床なし） : 15.40239910874552
床断熱住宅（浴室床断熱） : 15.40239910874552
床断熱住宅（浴室基礎断熱） : 15.40239910874552
基礎断熱住宅 : 15.40239910874552


## グルーピングされた熱橋（壁）

$$ L_{groupedHB,wall,SW} = L_{HB,ew-iw,SW} + L_{HB,ew-if,SW} $$  
$$ L_{groupedHB,wall,NW} = L_{HB,ew-iw,NW} + L_{HB,ew-if,NW} $$  
$$ L_{groupedHB,wall,NE} = L_{HB,ew-iw,NE} + L_{HB,ew-if,NE} $$  
$$ L_{groupedHB,wall,SE} = L_{HB,ew-iw,SE} + L_{HB,ew-if,SE} $$  

In [97]:
for h in hs:
    h.l_groupedhb_wall_sw = h.l_hb_ew_iw_sw + h.l_hb_ew_if_sw
    h.l_groupedhb_wall_nw = h.l_hb_ew_iw_nw + h.l_hb_ew_if_nw
    h.l_groupedhb_wall_ne = h.l_hb_ew_iw_ne + h.l_hb_ew_if_ne
    h.l_groupedhb_wall_se = h.l_hb_ew_iw_se + h.l_hb_ew_if_se

In [98]:
show_result(lambda h: (h.l_groupedhb_wall_sw, h.l_groupedhb_wall_nw, h.l_groupedhb_wall_ne, h.l_groupedhb_wall_se))

床断熱住宅（外皮に接する浴室床なし） : (13.894900835666089, 5.6, 5.6, 10.320073590817962)
床断熱住宅（浴室床断熱） : (13.894900835666089, 5.6, 5.6, 10.320073590817962)
床断熱住宅（浴室基礎断熱） : (13.894900835666089, 5.6, 5.6, 10.320073590817962)
基礎断熱住宅 : (13.894900835666089, 5.6, 5.6, 10.320073590817962)


## グルーピングされた熱橋（床）

$$ L_{groupedHB,floor,inside} = L_{HB,ef-iw,inside} + L_{HB,ef-base,inside} $$

In [99]:
for h in hs:
    h.l_groupedhb_floor_inside = h.l_hb_ef_iw_inside + h.l_hb_ef_base_inside

In [100]:
show_result(lambda h: h.l_groupedhb_floor_inside)

床断熱住宅（外皮に接する浴室床なし） : 19.04239910874552
床断熱住宅（浴室床断熱） : 19.04239910874552
床断熱住宅（浴室基礎断熱） : 19.04239910874552
基礎断熱住宅 : 15.40239910874552


## グルーピングされた熱橋（屋根又は天井と壁）

$$ L_{groupedHB,roof-wall,top-SW} = L_{HB,roof-ew,top-SW} + L_{HB,roof-ew-if,top-SW} $$  
$$ L_{groupedHB,roof-wall,top-NW} = L_{HB,roof-ew,top-NW} + L_{HB,roof-ew-if,top-NW} $$  
$$ L_{groupedHB,roof-wall,top-NE} = L_{HB,roof-ew,top-NE} + L_{HB,roof-ew-if,top-NE} $$  
$$ L_{groupedHB,roof-wall,top-SE} = L_{HB,roof-ew,top-SE} + L_{HB,roof-ew-if,top-SE} $$  

In [101]:
for h in hs:
    h.l_groupedhb_roof_wall_top_sw = h.l_hb_roof_ew_top_sw + h.l_hb_roof_ew_if_top_sw
    h.l_groupedhb_roof_wall_top_nw = h.l_hb_roof_ew_top_nw + h.l_hb_roof_ew_if_top_nw
    h.l_groupedhb_roof_wall_top_ne = h.l_hb_roof_ew_top_ne + h.l_hb_roof_ew_if_top_ne
    h.l_groupedhb_roof_wall_top_se = h.l_hb_roof_ew_top_se + h.l_hb_roof_ew_if_top_se

In [102]:
show_result(lambda h: (h.l_groupedhb_roof_wall_top_sw, h.l_groupedhb_roof_wall_top_nw, h.l_groupedhb_roof_wall_top_ne, h.l_groupedhb_roof_wall_top_se))

床断熱住宅（外皮に接する浴室床なし） : (10.609982280729707, 14.232564009651739, 27.199783952061885, 4.7924168280158135)
床断熱住宅（浴室床断熱） : (10.609982280729707, 14.232564009651739, 27.199783952061885, 4.7924168280158135)
床断熱住宅（浴室基礎断熱） : (10.609982280729707, 14.232564009651739, 27.199783952061885, 4.7924168280158135)
基礎断熱住宅 : (10.609982280729707, 14.232564009651739, 27.199783952061885, 4.7924168280158135)


## グルーピングされた熱橋（壁と壁）

$$ L_{groupedHB,wall-wall,SW-NW} = L_{HB,ew-ew,oc,SW-NW} + L_{HB,ew-ew,ic,SW-NW} $$  
$$ L_{groupedHB,wall-wall,NW-NE} = L_{HB,ew-ew,oc,NW-NE} + L_{HB,ew-ew,ic,NW-NE} $$  
$$ L_{groupedHB,wall-wall,NE-SE} = L_{HB,ew-ew,oc,NE-SE} + L_{HB,ew-ew,ic,NE-SE} $$  
$$ L_{groupedHB,wall-wall,SE-SW} = L_{HB,ew-ew,oc,SE-SW} + L_{HB,ew-ew,ic,SE-SW} $$  

In [103]:
for h in hs:
    h.l_groupedhb_wall_wall_sw_nw = h.l_hb_ew_ew_oc_sw_nw + h.l_hb_ew_ew_ic_sw_nw
    h.l_groupedhb_wall_wall_nw_ne = h.l_hb_ew_ew_oc_nw_ne + h.l_hb_ew_ew_ic_nw_ne
    h.l_groupedhb_wall_wall_ne_se = h.l_hb_ew_ew_oc_ne_se + h.l_hb_ew_ew_ic_ne_se
    h.l_groupedhb_wall_wall_se_sw = h.l_hb_ew_ew_oc_se_sw + h.l_hb_ew_ew_ic_se_sw

In [104]:
show_result(lambda h: (h.l_groupedhb_wall_wall_sw_nw, h.l_groupedhb_wall_wall_nw_ne, h.l_groupedhb_wall_wall_ne_se, h.l_groupedhb_wall_wall_se_sw))

床断熱住宅（外皮に接する浴室床なし） : (5.6, 17.2, 5.6, 5.6)
床断熱住宅（浴室床断熱） : (5.6, 17.2, 5.6, 5.6)
床断熱住宅（浴室基礎断熱） : (5.6, 17.2, 5.6, 5.6)
基礎断熱住宅 : (5.6, 17.2, 5.6, 5.6)


## グルーピングされた熱橋（壁と床）

$$ L_{groupedHB,wall-floor,SW-bottom} = L_{HB,ef-ew-base,SW-bottom} $$  
$$ L_{groupedHB,wall-floor,NW-bottom} = L_{HB,ef-ew-base,NW-bottom} $$  
$$ L_{groupedHB,wall-floor,NE-bottom} = L_{HB,ef-ew-base,NE-bottom} $$  
$$ L_{groupedHB,wall-floor,SE-bottom} = L_{HB,ef-ew-base,SE-bottom} $$  

In [105]:
for h in hs:
    h.l_groupedhb_wall_floor_sw_bottom = h.l_hb_ef_ew_base_sw_bottom
    h.l_groupedhb_wall_floor_nw_bottom = h.l_hb_ef_ew_base_nw_bottom
    h.l_groupedhb_wall_floor_ne_bottom = h.l_hb_ef_ew_base_ne_bottom
    h.l_groupedhb_wall_floor_se_bottom = h.l_hb_ef_ew_base_se_bottom

In [106]:
show_result(lambda h: (h.l_groupedhb_wall_floor_sw_bottom, h.l_groupedhb_wall_floor_nw_bottom, h.l_groupedhb_wall_floor_ne_bottom, h.l_groupedhb_wall_floor_se_bottom))

床断熱住宅（外皮に接する浴室床なし） : (10.609982280729707, 2.972416828015813, 9.244982280729706, 4.7924168280158135)
床断熱住宅（浴室床断熱） : (10.609982280729707, 2.972416828015813, 9.244982280729706, 4.7924168280158135)
床断熱住宅（浴室基礎断熱） : (10.609982280729707, 2.972416828015813, 9.244982280729706, 4.7924168280158135)
基礎断熱住宅 : (10.609982280729707, 2.972416828015813, 9.244982280729706, 4.7924168280158135)


# 数字の丸め

## 外皮の部位の面積の合計

In [107]:
for h, doc in zip(hs, docs):
    doc['外皮の部位の面積の合計'] = round2(h.a_env)

## 床面積の合計

In [108]:
for h, doc in zip(hs, docs):
    doc['床面積の合計'] = round2(h.a_f_total)

## 屋根又は天井の面積

In [109]:
for h, doc in zip(hs, docs):
    doc['屋根又は天井の面積'] = round2(h.a_roof)

## 壁の面積 

In [110]:
for h, doc in zip(hs, docs):
    doc['南西に面した壁の面積'] = round2(h.a_wall[0])
    doc['北西に面した壁の面積'] = round2(h.a_wall[1])
    doc['北東に面した壁の面積'] = round2(h.a_wall[2])
    doc['南東に面した壁の面積'] = round2(h.a_wall[3])

## ドアの面積

In [111]:
for h, doc in zip(hs, docs):
    doc['南西に面したドアの面積'] = round2(h.a_door[0])
    doc['北西に面したドアの面積'] = round2(h.a_door[1])
    doc['北東に面したドアの面積'] = round2(h.a_door[2])
    doc['南東に面したドアの面積'] = round2(h.a_door[3])

## 窓の面積

In [112]:
for h, doc in zip(hs, docs):
    doc['南西に面した窓の面積'] = round2(h.a_wnd[0])
    doc['北西に面した窓の面積'] = round2(h.a_wnd[1])
    doc['北東に面した窓の面積'] = round2(h.a_wnd[2])
    doc['南東に面した窓の面積'] = round2(h.a_wnd[3])

## 浴室の床の面積

In [113]:
for h, doc in zip(hs, docs):
    doc['浴室の床の面積'] = round2(h.a_if_bath)

## その他の床の面積

In [114]:
for h, doc in zip(hs, docs):
    doc['その他の床の面積'] = round2(h.a_if_other)

## 玄関等の基礎の面積

In [115]:
for h, doc in zip(hs, docs):
    doc['南西に面した玄関等の基礎の面積'] = round2(h.a_base_entrance_outside[0])
    doc['北西に面した玄関等の基礎の面積'] = round2(h.a_base_entrance_outside[1])
    doc['北東に面した玄関等の基礎の面積'] = round2(h.a_base_entrance_outside[2])
    doc['南東に面した玄関等の基礎の面積'] = round2(h.a_base_entrance_outside[3])
    doc['床下に面した玄関等の基礎の面積'] = round2(h.a_base_entrance_inside)

## 浴室の基礎の面積

In [116]:
for h, doc in zip(hs, docs):
    doc['南西に面した浴室の基礎の面積'] = round2(h.a_base_bath_outside[0])
    doc['北西に面した浴室の基礎の面積'] = round2(h.a_base_bath_outside[1])
    doc['北東に面した浴室の基礎の面積'] = round2(h.a_base_bath_outside[2])
    doc['南東に面した浴室の基礎の面積'] = round2(h.a_base_bath_outside[3])
    doc['床下に面した浴室の基礎の面積'] = round2(h.a_base_bath_inside)

## その他の基礎の面積

In [117]:
for h, doc in zip(hs, docs):
    doc['南西に面したその他の基礎の面積'] = round2(h.a_base_other_outside[0])
    doc['北西に面したその他の基礎の面積'] = round2(h.a_base_other_outside[1])
    doc['北東に面したその他の基礎の面積'] = round2(h.a_base_other_outside[2])
    doc['南東に面したその他の基礎の面積'] = round2(h.a_base_other_outside[3])
    doc['床下に面したその他の基礎の面積'] = round2(h.a_base_other_inside)

## 玄関等の土間床等の外周部の長さ

In [118]:
for h, doc in zip(hs, docs):
    doc['南西に面した玄関等の土間床等の外周部の長さ'] = round2(h.l_base_entrance_outside[0])
    doc['北西に面した玄関等の土間床等の外周部の長さ'] = round2(h.l_base_entrance_outside[1])
    doc['北東に面した玄関等の土間床等の外周部の長さ'] = round2(h.l_base_entrance_outside[2])
    doc['南東に面した玄関等の土間床等の外周部の長さ'] = round2(h.l_base_entrance_outside[3])
    doc['床下に面した玄関等の土間床等の外周部の長さ'] = round2(h.l_base_entrance_inside)

## 浴室の土間床等の外周部の長さ

In [119]:
for h, doc in zip(hs, docs):
    doc['南西に面した浴室の土間床等の外周部の長さ'] = round2(h.l_base_bath_outside[0])
    doc['北西に面した浴室の土間床等の外周部の長さ'] = round2(h.l_base_bath_outside[1])
    doc['北東に面した浴室の土間床等の外周部の長さ'] = round2(h.l_base_bath_outside[2])
    doc['南東に面した浴室の土間床等の外周部の長さ'] = round2(h.l_base_bath_outside[3])
    doc['床下に面した浴室の土間床等の外周部の長さ'] = round2(h.l_base_bath_inside)

## その他の土間床等の外周部の長さ

In [120]:
for h, doc in zip(hs, docs):
    doc['南西に面したその他の土間床等の外周部の長さ'] = round2(h.l_base_other_outside[0])
    doc['北西に面したその他の土間床等の外周部の長さ'] = round2(h.l_base_other_outside[1])
    doc['北東に面したその他の土間床等の外周部の長さ'] = round2(h.l_base_other_outside[2])
    doc['南東に面したその他の土間床等の外周部の長さ'] = round2(h.l_base_other_outside[3])
    doc['床下に面したその他の土間床等の外周部の長さ'] = round2(h.l_base_other_inside)

## 屋根又は天井の熱橋

In [121]:
for h, doc in zip(hs, docs):
    doc['上面に面した屋根又は天井の熱橋'] = round2(h.l_groupedhb_roof_top)

## 壁の熱橋

In [122]:
for h, doc in zip(hs, docs):
    doc['南西に面した壁の熱橋'] = round2(h.l_groupedhb_wall_sw)
    doc['北西に面した壁の熱橋'] = round2(h.l_groupedhb_wall_nw)
    doc['北東に面した壁の熱橋'] = round2(h.l_groupedhb_wall_ne)
    doc['南東に面した壁の熱橋'] = round2(h.l_groupedhb_wall_se)

## 床の熱橋

In [123]:
for h, doc in zip(hs, docs):
    doc['床下に面した床の熱橋'] = round2(h.l_groupedhb_floor_inside)

## 屋根又は天井と壁の熱橋

In [124]:
for h, doc in zip(hs, docs):
    doc['上面と南西に面した屋根又は天井と壁の熱橋'] = round2(h.l_groupedhb_roof_wall_top_sw)
    doc['上面と北西に面した屋根又は天井と壁の熱橋'] = round2(h.l_groupedhb_roof_wall_top_nw)
    doc['上面と北東に面した屋根又は天井と壁の熱橋'] = round2(h.l_groupedhb_roof_wall_top_ne)
    doc['上面と南東に面した屋根又は天井と壁の熱橋'] = round2(h.l_groupedhb_roof_wall_top_se)

## 壁と壁の熱橋

In [125]:
for h, doc in zip(hs, docs):
    doc['南西と北西に面した壁と壁の熱橋'] = round2(h.l_groupedhb_wall_wall_sw_nw)
    doc['北西と北東に面した壁と壁の熱橋'] = round2(h.l_groupedhb_wall_wall_nw_ne)
    doc['北東と南東に面した壁と壁の熱橋'] = round2(h.l_groupedhb_wall_wall_ne_se)
    doc['南東と南西に面した壁と壁の熱橋'] = round2(h.l_groupedhb_wall_wall_se_sw)

## 壁と床の熱橋

In [126]:
for h, doc in zip(hs, docs):
    doc['南西と床下に面した壁と床の熱橋'] = round2(h.l_groupedhb_wall_floor_sw_bottom)
    doc['北西と床下に面した壁と床の熱橋'] = round2(h.l_groupedhb_wall_floor_nw_bottom)
    doc['北東と床下に面した壁と床の熱橋'] = round2(h.l_groupedhb_wall_floor_ne_bottom)
    doc['南東と床下に面した壁と床の熱橋'] = round2(h.l_groupedhb_wall_floor_se_bottom)

# 簡易法の係数

In [127]:
def roundup(v: float, d: int) -> float:
    TWOPLACES = Decimal(10) ** -d
    output = Decimal(str(v)).quantize(TWOPLACES, rounding=ROUND_UP)
    return float(output)

In [128]:
def rounddown(v: float, d: int) -> float:
    TWOPLACES = Decimal(10) ** -d
    output = Decimal(str(v)).quantize(TWOPLACES, rounding=ROUND_DOWN)
    return float(output)

## UA値を求める係数

$$ C_{UA,roof} = \frac{A_{roof}}{A_{env}} $$

$$ C_{UA,wall} = \frac{A_{wall,SW}+A_{wall,NW}+A_{wall,NE}+A_{wall,SE}}{A_{env}} $$

$$ C_{UA,door} = \frac{A_{door,SW}+A_{door,NW}+A_{door,NE}+A_{door,SE}}{A_{env}} $$

$$ C_{UA,wnd} = \frac{A_{wnd,SW}+A_{wnd,NW}+A_{wnd,NE}+A_{wnd,SE}}{A_{env}} $$

$$ C_{UA,if,bath} = \frac{A_{if,bath}}{A_{env}} \times 0.7 $$

$$ C_{UA,if,other} = \frac{A_{if,other}}{A_{env}} \times 0.7 $$

$$ C_{UA,base,entrance} = \frac{A_{base,entrance,SW}+A_{base,entrance,NW}+A_{base,entrance,NE}+A_{base,entrance,SE}+A_{base,entrance,inside} \times 0.7}{A_{env}} $$

$$ C_{UA,base,bath} = \frac{A_{base,bath,SW}+A_{base,bath,NW}+A_{base,bath,NE}+A_{base,bath,SE}+A_{base,bath,inside} \times 0.7}{A_{env}} $$

$$ C_{UA,base,other} = \frac{A_{base,other,SW}+A_{base,other,NW}+A_{base,other,NE}+A_{base,other,SE}+A_{base,other,inside} \times 0.7}{A_{env}} $$

$$ C_{UA,ef,entrance} = \frac{l_{base,entrance,SW}+l_{base,entrance,NW}+l_{base,entrance,NE}+l_{base,entrance,SE}+l_{base,entrance,inside} \times 0.7}{A_{env}} $$

$$ C_{UA,ef,bath} = \frac{l_{base,bath,SW}+l_{base,bath,NW}+l_{base,bath,NE}+l_{base,bath,SE}+l_{base,bath,inside} \times 0.7}{A_{env}} $$

$$ C_{UA,ef,other} = \frac{l_{base,other,SW}+l_{base,other,NW}+l_{base,other,NE}+l_{base,other,SE}+l_{base,other,inside} \times 0.7}{A_{env}} $$


In [129]:
for h, doc in zip(hs, docs):
    doc['UA係数 屋根・天井'] = roundup(doc['屋根又は天井の面積'] / doc['外皮の部位の面積の合計'], 3)
    doc['UA係数 壁'] = roundup((
        doc['南西に面した壁の面積'] + doc['北西に面した壁の面積'] + doc['北東に面した壁の面積'] + doc['南東に面した壁の面積']
    ) / doc['外皮の部位の面積の合計'], 3)
    doc['UA係数 ドア'] = roundup((
        doc['南西に面したドアの面積'] + doc['北西に面したドアの面積'] + doc['北東に面したドアの面積'] + doc['南東に面したドアの面積']
    ) / doc['外皮の部位の面積の合計'], 3)
    doc['UA係数 窓'] = roundup((
        doc['南西に面した窓の面積'] + doc['北西に面した窓の面積'] + doc['北東に面した窓の面積'] + doc['南東に面した窓の面積']
    ) / doc['外皮の部位の面積の合計'], 3)
    doc['UA係数 床（浴室）'] = roundup(doc['浴室の床の面積'] / doc['外皮の部位の面積の合計'] * 0.7, 3)
    doc['UA係数 床（その他）'] = roundup(doc['その他の床の面積'] / doc['外皮の部位の面積の合計'] * 0.7, 3)
    doc['UA係数 基礎（玄関）'] = roundup((
        doc['南西に面した玄関等の基礎の面積'] + doc['北西に面した玄関等の基礎の面積']
        + doc['北東に面した玄関等の基礎の面積'] + doc['南東に面した玄関等の基礎の面積']
        + doc['床下に面した玄関等の基礎の面積'] * 0.7) / doc['外皮の部位の面積の合計'], 3)
    doc['UA係数 基礎（浴室その他）'] = roundup((
        doc['南西に面した浴室の基礎の面積'] + doc['北西に面した浴室の基礎の面積']
        + doc['北東に面した浴室の基礎の面積'] + doc['南東に面した浴室の基礎の面積']
        + doc['床下に面した浴室の基礎の面積'] * 0.7
        + doc['南西に面したその他の基礎の面積'] + doc['北西に面したその他の基礎の面積']
        + doc['北東に面したその他の基礎の面積'] + doc['南東に面したその他の基礎の面積']
        + doc['床下に面したその他の基礎の面積'] * 0.7) / doc['外皮の部位の面積の合計'], 3)
    doc['UA係数 土間（玄関）'] = roundup((
        doc['南西に面した玄関等の土間床等の外周部の長さ'] + doc['北西に面した玄関等の土間床等の外周部の長さ']
        + doc['北東に面した玄関等の土間床等の外周部の長さ'] + doc['南東に面した玄関等の土間床等の外周部の長さ']
        + doc['床下に面した玄関等の土間床等の外周部の長さ'] * 0.7) / doc['外皮の部位の面積の合計'], 3)
    doc['UA係数 土間（浴室その他）'] = roundup((
        doc['南西に面した浴室の土間床等の外周部の長さ'] + doc['北西に面した浴室の土間床等の外周部の長さ']
        + doc['北東に面した浴室の土間床等の外周部の長さ'] + doc['南東に面した浴室の土間床等の外周部の長さ']
        + doc['床下に面した浴室の土間床等の外周部の長さ'] * 0.7
        + doc['南西に面したその他の土間床等の外周部の長さ'] + doc['北西に面したその他の土間床等の外周部の長さ']
        + doc['北東に面したその他の土間床等の外周部の長さ'] + doc['南東に面したその他の土間床等の外周部の長さ']
        + doc['床下に面したその他の土間床等の外周部の長さ'] * 0.7) / doc['外皮の部位の面積の合計'], 3)

## ηA値を求める係数

In [130]:
f = f_value.f1f2Value()

$$ C_{UA,roof} = \frac{A_{roof}}{A_{env}} $$

$$ C_{UA,wall} = \frac{A_{wall,SW}+A_{wall,NW}+A_{wall,NE}+A_{wall,SE}}{A_{env}} $$

$$ C_{UA,door} = \frac{A_{door,SW}+A_{door,NW}+A_{door,NE}+A_{door,SE}}{A_{env}} $$

$$ C_{UA,wnd} = \frac{A_{wnd,SW}+A_{wnd,NW}+A_{wnd,NE}+A_{wnd,SE}}{A_{env}} $$

$$ C_{UA,if,bath} = \frac{A_{if,bath}}{A_{env}} \times 0.7 $$

$$ C_{UA,if,other} = \frac{A_{if,other}}{A_{env}} \times 0.7 $$

$$ C_{UA,base,entrance} = \frac{A_{base,entrance,SW}+A_{base,entrance,NW}+A_{base,entrance,NE}+A_{base,entrance,SE}+A_{base,entrance,inside} \times 0.7}{A_{env}} $$

$$ C_{UA,base,bath} = \frac{A_{base,bath,SW}+A_{base,bath,NW}+A_{base,bath,NE}+A_{base,bath,SE}+A_{base,bath,inside} \times 0.7}{A_{env}} $$

$$ C_{UA,base,other} = \frac{A_{base,other,SW}+A_{base,other,NW}+A_{base,other,NE}+A_{base,other,SE}+A_{base,other,inside} \times 0.7}{A_{env}} $$

$$ C_{UA,ef,entrance} = \frac{l_{base,entrance,SW}+l_{base,entrance,NW}+l_{base,entrance,NE}+l_{base,entrance,SE}+l_{base,entrance,inside} \times 0.7}{A_{env}} $$

$$ C_{UA,ef,bath} = \frac{l_{base,bath,SW}+l_{base,bath,NW}+l_{base,bath,NE}+l_{base,bath,SE}+l_{base,bath,inside} \times 0.7}{A_{env}} $$

$$ C_{UA,ef,other} = \frac{l_{base,other,SW}+l_{base,other,NW}+l_{base,other,NE}+l_{base,other,SE}+l_{base,other,inside} \times 0.7}{A_{env}} $$


In [131]:
for h, doc in zip(hs, docs):
    for region in [1,2,3,4,5,6,7,8]:
        doc['ηAC係数 屋根・天井 ' + str(region) + '地域'] = roundup(
            doc['屋根又は天井の面積'] * dc.get_nu_c(region=region, direction='top') * 0.034 * 100 / doc['外皮の部位の面積の合計'], 3)
        doc['ηAC係数 壁 ' + str(region) + '地域'] = roundup((
            doc['南西に面した壁の面積'] * dc.get_nu_c(region=region, direction='sw') * 0.034
            + doc['北西に面した壁の面積'] * dc.get_nu_c(region=region, direction='nw') * 0.034
            + doc['北東に面した壁の面積'] * dc.get_nu_c(region=region, direction='ne') * 0.034
            + doc['南東に面した壁の面積'] * dc.get_nu_c(region=region, direction='se') * 0.034
        ) / doc['外皮の部位の面積の合計'] * 100, 3)
        doc['ηAC係数 ドア ' + str(region) + '地域'] = roundup((
            doc['南西に面したドアの面積'] * dc.get_nu_c(region=region, direction='sw') * 0.034
            + doc['北西に面したドアの面積'] * dc.get_nu_c(region=region, direction='nw') * 0.034
            + doc['北東に面したドアの面積'] * dc.get_nu_c(region=region, direction='ne') * 0.034
            + doc['南東に面したドアの面積'] * dc.get_nu_c(region=region, direction='se') * 0.034
        ) / doc['外皮の部位の面積の合計'] * 100, 3)
        doc['ηAC係数 窓 ' + str(region) + '地域'] = roundup((
            doc['南西に面した窓の面積'] * dc.get_nu_c(region=region, direction='sw')
                * f.getValue(glassType='Type1', season='Cooling', region='region'+str(region), direction='SW', lValue=20.0)
            + doc['北西に面した窓の面積'] * dc.get_nu_c(region=region, direction='nw')
                * f.getValue(glassType='Type1', season='Cooling', region='region'+str(region), direction='NW', lValue=20.0)
            + doc['北東に面した窓の面積'] * dc.get_nu_c(region=region, direction='ne')
                * f.getValue(glassType='Type1', season='Cooling', region='region'+str(region), direction='NE', lValue=20.0)
            + doc['南東に面した窓の面積'] * dc.get_nu_c(region=region, direction='se')
                * f.getValue(glassType='Type1', season='Cooling', region='region'+str(region), direction='SE', lValue=20.0)
        ) / doc['外皮の部位の面積の合計'] * 100, 3)
        doc['ηAC係数 玄関等の基礎 ' + str(region) + '地域'] = roundup((
            doc['南西に面した玄関等の基礎の面積'] * dc.get_nu_c(region=region, direction='sw') * 0.034
            + doc['北西に面した玄関等の基礎の面積'] * dc.get_nu_c(region=region, direction='nw') * 0.034
            + doc['北東に面した玄関等の基礎の面積'] * dc.get_nu_c(region=region, direction='ne') * 0.034
            + doc['南東に面した玄関等の基礎の面積'] * dc.get_nu_c(region=region, direction='se') * 0.034
        ) / doc['外皮の部位の面積の合計'] * 100, 3)
        doc['ηAC係数 浴室その他の基礎 ' + str(region) + '地域'] = roundup((
            doc['南西に面した浴室の基礎の面積'] * dc.get_nu_c(region=region, direction='sw') * 0.034
            + doc['北西に面した浴室の基礎の面積'] * dc.get_nu_c(region=region, direction='nw') * 0.034
            + doc['北東に面した浴室の基礎の面積'] * dc.get_nu_c(region=region, direction='ne') * 0.034
            + doc['南東に面した浴室の基礎の面積'] * dc.get_nu_c(region=region, direction='se') * 0.034
            + doc['南西に面したその他の基礎の面積'] * dc.get_nu_c(region=region, direction='sw') * 0.034
            + doc['北西に面したその他の基礎の面積'] * dc.get_nu_c(region=region, direction='nw') * 0.034
            + doc['北東に面したその他の基礎の面積'] * dc.get_nu_c(region=region, direction='ne') * 0.034
            + doc['南東に面したその他の基礎の面積'] * dc.get_nu_c(region=region, direction='se') * 0.034
        ) / doc['外皮の部位の面積の合計'] * 100, 3)
    for region in [1,2,3,4,5,6,7]:
        doc['ηAH係数 屋根・天井 ' + str(region) + '地域'] = rounddown(
            doc['屋根又は天井の面積'] * dc.get_nu_h(region=region, direction='top') * 0.034 * 100 / doc['外皮の部位の面積の合計'], 3)
        doc['ηAH係数 壁 ' + str(region) + '地域'] = rounddown((
            doc['南西に面した壁の面積'] * dc.get_nu_h(region=region, direction='sw') * 0.034 * 100
            + doc['北西に面した壁の面積'] * dc.get_nu_h(region=region, direction='nw') * 0.034 * 100
            + doc['北東に面した壁の面積'] * dc.get_nu_h(region=region, direction='ne') * 0.034 * 100
            + doc['南東に面した壁の面積'] * dc.get_nu_h(region=region, direction='se') * 0.034 * 100
        ) / doc['外皮の部位の面積の合計'], 3)
        doc['ηAH係数 ドア ' + str(region) + '地域'] = rounddown((
            doc['南西に面したドアの面積'] * dc.get_nu_h(region=region, direction='sw') * 0.034 * 100
            + doc['北西に面したドアの面積'] * dc.get_nu_h(region=region, direction='nw') * 0.034 * 100
            + doc['北東に面したドアの面積'] * dc.get_nu_h(region=region, direction='ne') * 0.034 * 100
            + doc['南東に面したドアの面積'] * dc.get_nu_h(region=region, direction='se') * 0.034 * 100
        ) / doc['外皮の部位の面積の合計'], 3)
        doc['ηAH係数 窓 ' + str(region) + '地域'] = rounddown((
            doc['南西に面した窓の面積'] * dc.get_nu_h(region=region, direction='sw') * 100
                * f.getValue(glassType='Type7', season='Heating', region='region'+str(region), direction='SW', lValue=1/0.3)
            + doc['北西に面した窓の面積'] * dc.get_nu_h(region=region, direction='nw') * 100
                * f.getValue(glassType='Type7', season='Heating', region='region'+str(region), direction='NW', lValue=1/0.3)
            + doc['北東に面した窓の面積'] * dc.get_nu_h(region=region, direction='ne') * 100
                * f.getValue(glassType='Type7', season='Heating', region='region'+str(region), direction='NE', lValue=1/0.3)
            + doc['南東に面した窓の面積'] * dc.get_nu_h(region=region, direction='se') * 100
                * f.getValue(glassType='Type7', season='Heating', region='region'+str(region), direction='SE', lValue=1/0.3)
        ) / doc['外皮の部位の面積の合計'], 3)
        doc['ηAH係数 玄関等の基礎 ' + str(region) + '地域'] = rounddown((
            doc['南西に面した玄関等の基礎の面積'] * dc.get_nu_h(region=region, direction='sw') * 0.034 * 100
            + doc['北西に面した玄関等の基礎の面積'] * dc.get_nu_h(region=region, direction='nw') * 0.034 * 100
            + doc['北東に面した玄関等の基礎の面積'] * dc.get_nu_h(region=region, direction='ne') * 0.034 * 100
            + doc['南東に面した玄関等の基礎の面積'] * dc.get_nu_h(region=region, direction='se') * 0.034 * 100
        ) / doc['外皮の部位の面積の合計'], 3)
        doc['ηAH係数 浴室その他の基礎 ' + str(region) + '地域'] = rounddown((
            doc['南西に面した浴室の基礎の面積'] * dc.get_nu_h(region=region, direction='sw') * 0.034 * 100
            + doc['北西に面した浴室の基礎の面積'] * dc.get_nu_h(region=region, direction='nw') * 0.034 * 100
            + doc['北東に面した浴室の基礎の面積'] * dc.get_nu_h(region=region, direction='ne') * 0.034 * 100
            + doc['南東に面した浴室の基礎の面積'] * dc.get_nu_h(region=region, direction='se') * 0.034 * 100
            + doc['南西に面したその他の基礎の面積'] * dc.get_nu_h(region=region, direction='sw') * 0.034 * 100
            + doc['北西に面したその他の基礎の面積'] * dc.get_nu_h(region=region, direction='nw') * 0.034 * 100
            + doc['北東に面したその他の基礎の面積'] * dc.get_nu_h(region=region, direction='ne') * 0.034 * 100
            + doc['南東に面したその他の基礎の面積'] * dc.get_nu_h(region=region, direction='se') * 0.034 * 100
        ) / doc['外皮の部位の面積の合計'], 3)

# テスト

In [132]:
wb = xlrd.open_workbook('publicated_value.xlsx')
sheet = wb.sheet_by_name('test2020')
values = [sheet.row_values(row) for row in range(sheet.nrows)]

In [133]:
n_OK = 0
n = 0
for i, value in enumerate(values):
    if i != 0:
        is_ok = True
        for j, doc in enumerate(docs):
            real = doc[value[0]]
            expected = value[j+1]
            if abs(real - expected) > 0.000001:
                print(str(value[0]) + ': house ' + str(j) + ' error: real = ' + str(real) + ' expected = ' + str(expected) )
                is_ok = False
        if is_ok:
            n_OK = n_OK + 1
        n = n+1        

print('テスト数: ' + str(n))
print('テスト通過数: ' + str(n_OK))

テスト数: 165
テスト通過数: 165
